In [8]:
import pandas as pd
import json
import time
from google_currency import convert 


In [6]:
df=pd.read_csv('/Users/mikel/Documents/Projects/chameleon-pricing/country_currency.csv')
df.head()

,Country,last_value,Currency,Code
0,aruba,1.350690,Aruban Florin,AWG
1,afghanistan,17.914483,Afghani,AFN
2,angola,146.167168,Kwanza,AOA
3,albania,41.342391,Lek,ALL
4,united arab emirates,2.258503,UAE Dirham,AED


# PPP to dollar

https://pypi.org/project/google-currency/

!pip install google-currency

In [10]:
def ppp_to_dollar(Code, last_value):
    conversion=convert(Code, 'usd', last_value)
    result = json.loads(conversion)
    return result['amount']

In [11]:
ppp_to_dollar('BRL', 0.624484)

'0.12'

In [12]:
%%time
apply = df.apply(
         lambda x: ppp_to_dollar(x['Code'], x['last_value']),
         axis=1)
df['ppp_todollar']  = apply
print(df)

Invalid currency codes passed in parameters, original exception message is -> 'BOV'
Invalid currency codes passed in parameters, original exception message is -> 'CHE'
Invalid currency codes passed in parameters, original exception message is -> 'CHW'
Invalid currency codes passed in parameters, original exception message is -> 'COU'
Invalid currency codes passed in parameters, original exception message is -> 'CUC'
Unable to convert currency, failed to fetch results from Google
Invalid currency codes passed in parameters, original exception message is -> 'ERN'
Invalid currency codes passed in parameters, original exception message is -> 'GHS'
Invalid currency codes passed in parameters, original exception message is -> 'MXV'
Invalid currency codes passed in parameters, original exception message is -> 'MNT'
Invalid currency codes passed in parameters, original exception message is -> 'MZN'
Invalid currency codes passed in parameters, original exception message is -> 'RON'
Invalid curr

In [13]:
df.head()

,Country,last_value,Currency,Code,ppp_todollar
0,aruba,1.350690,Aruban Florin,AWG,0.75
1,afghanistan,17.914483,Afghani,AFN,0.23
2,angola,146.167168,Kwanza,AOA,0.23
3,albania,41.342391,Lek,ALL,0.41
4,united arab emirates,2.258503,UAE Dirham,AED,0.61


In [14]:
df.dtypes

Country          object
last_value      float64
Currency         object
Code             object
ppp_todollar     object
dtype: object

In [15]:
df["ppp_todollar"] = df.ppp_todollar.astype(float)

In [16]:
df.dtypes

Country          object
last_value      float64
Currency         object
Code             object
ppp_todollar    float64
dtype: object

In [17]:
df[df.ppp_todollar == 0.00]

,Country,last_value,Currency,Code,ppp_todollar
23,bolivia,2.694105,Mvdol,BOV,0.0
32,switzerland,1.143836,WIR Euro,CHE,0.0
34,switzerland,1.143836,WIR Franc,CHW,0.0
40,colombia,1351.775746,Unidad de Valor Real,COU,0.0
44,cuba,0.321939,Peso Convertible,CUC,0.0
49,germany,0.744679,Euro,EUR,0.0
56,eritrea,5.469904,Nakfa,ERN,0.0
66,ghana,2.032734,Ghana Cedi,GHS,0.0
107,mexico,9.505202,Mexican Unidad de Inversion (UDI),MXV,0.0
113,mongolia,898.732634,Tugrik,MNT,0.0


In [18]:
df.drop(df[df['ppp_todollar'] == 0.0].index, inplace = True)

# Adjustment function

In [19]:
def price_calculator(product_price, country):
    find_ppp=df.ppp_todollar[df.Country == country.lower()]
    return product_price * find_ppp

In [22]:
price_calculator(60, 'colombia')

39    22.2
Name: ppp_todollar, dtype: float64

In [24]:
df.to_excel('/Users/mikel/Documents/Projects/chameleon-pricing/DB.xlsx', index=False)